In [1]:
using SparseArrays 

# 读取数据
nodes = CSV.read("names.csv", DataFrame)
edges = CSV.read("edges.csv", DataFrame)

# 转换成矩阵
nodes = Matrix(nodes)
edges = Matrix(edges)

# 构建稀疏邻接矩阵
n = size(nodes, 1)
A = sparse(
    edges[2:end, 1],   # row indices
    edges[2:end, 2],   # column indices
    fill(1.0, size(edges, 1)-1),  # edge weights
    n,             # number of rows
    n              # number of columns
)
A = transpose(A)

# 构建转移矩阵
row_sums = vec(sum(A, dims=1))
probs = ifelse.(row_sums .== 0, 0.0, 1 ./ row_sums)
diag_matrix = spdiagm(0 => probs)
T = A * diag_matrix

# Power Method 计算 PageRank
function power_iteration(A, T, x)
    eps = 1e-15  # tolerance for convergence
    max_iter = 1000  # maximum number of iterations
    iter = 0
    
    while iter < max_iter
        x_new = T * x
        if norm(A * diag_matrix * x_new - x_new, 1) < eps
            break
        end
        
        x = x_new
        iter += 1
    end
    
    return x
end

# 计算 PageRank
x = ones(n) / n
pr = power_iteration(A, T, x)

# 输出排名前 10 的页面
sorted_pr = sort(pr, rev=true)
top10_idx = sorted_pr[1:10]
for i in top10_idx
    println(nodes[i, 1])
end


UndefVarError: UndefVarError: CSV not defined